In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/MyDrive/atlantic-hurricane-trajectory-prediction/docs

In [ ]:
!ls

In [ ]:
%autosave 60

In [ ]:
# Import various libraries throughout the software
from pprint import pprint
from datetime import timedelta
from sklearn.preprocessing import RobustScaler
from geopy.distance import great_circle as vc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as Math
import datetime
import dateutil
import io

In [ ]:
# data cleaning/processing: (from hurricane-net, hammad)
db = []
with open('data/hurdat2-1851-2022-050423.txt') as raw: 
    for line in raw: 
        line = line.replace(' ', '').split(',')
    
        # Identify atlantic storm, first 2 letters should be AL
        if (line[0][:2] == 'AL') :
            storm_id = line[0]
            storm_name = line[1]
            storm_entries = line[2]

            # Iterate and read through best track entries
            for i in range(int(storm_entries)) :
                entry = raw.readline().replace(' ', '').split(',')
                # Filter -999 placeholder for missing central pressure
                entry = [None if x == "-999" else x for x in entry]
                # Construct date and time based on first two columns
                timestamp = datetime.datetime(int(entry[0][:4]), int(entry[0][4:6]), int(entry[0][6:8]), int(entry[1][:2]), int(entry[1][3:]))
                # Add entry into our current database
                db.append([storm_id, storm_name, timestamp] + entry[2:-1])
        else :
            print("Error, unidentified storm ".join(str(line[0])))

# Return DataFrame
dataset = pd.DataFrame(db, columns = ['storm_id', 'storm_name', 'entry_time', 'entry_id', 'entry_status', 'lat', 'long','max_wind', 'min_pressure', '34kt_ne', '34kt_se', '34kt_sw', '34kt_nw', '50kt_ne', '50kt_se', '50kt_sw', '50kt_nw', '64kt_ne', '64kt_se', '64kt_sw', '64kt_nw'])

In [ ]:
models = dict()
class model :
  '''
  PURPOSE: To create a class for each model included in the forecast error database
  METHOD: Provide an API
  OUTOUT: A class with a DataFrame and associated operations
  '''
  name = None
  # Dictionary key: STMID
  storm = dict()
  def __init__(self, model_name) :
    self.name = model_name
    return

with open('errors/1970-present_OFCL_v_BCD5_ind_ATL_TI_errors_noTDs.txt') as raw :
    lines = raw.readlines()
    
    # Get model names and declare model objects
    line = lines[1].split()
    model_names = line[2:]
    for model_name in model_names :
        models[model_name] = model(model_name)
    
    # Data starts at line 9 
    for line in lines[9:] :
        line = line.split()
        # Identify atlantic storm date, storm id, associated sample sizes, latitude and longitude, and windspeed
        timestamp = datetime.datetime.strptime(line[0], "%d-%m-%Y/%H:%M:%S")
        storm_id = line[1]
        sample_sizes = {"F012": float(line[2]), "F024": float(line[3]),"F036": float(line[4]), "F048": float(line[5]), "F072": float(line[6]), "F096": float(line[7]), "F120": float(line[8]), "F144": float(line[9]), "F168": float(line[10])} 
        latitude = float(line[11])
        longitude = float(line[12])
        wind_speed = float(line[13])
    
                
        # Iterate through model forecast track and intensity errors 
        for i in range(len(model_names)) :
            intensity_forecast = dict(list(zip([timestamp, timestamp + timedelta(hours = 12), timestamp + timedelta(hours = 24), timestamp + timedelta(hours = 36), timestamp + timedelta(hours = 48), timestamp + timedelta(hours = 72), timestamp + timedelta(hours = 96), timestamp + timedelta(hours = 120), timestamp + timedelta(hours = 144), timestamp + timedelta(hours = 168)], [None if x == "-9999.0" else float(x) for x in line[14 + (20 * i) : 24 + (20 * i)]])))
            track_forecast = dict(list(zip([timestamp, timestamp + timedelta(hours = 12), timestamp + timedelta(hours = 24), timestamp + timedelta(hours = 36), timestamp + timedelta(hours = 48), timestamp + timedelta(hours = 72), timestamp + timedelta(hours = 96), timestamp + timedelta(hours = 120), timestamp + timedelta(hours = 144), timestamp + timedelta(hours = 168)], [None if x == "-9999.0" else float(x) for x in line[24 + (20 * i) : 34 + (20 * i)]])))
        
        # Add forecast to model and storm, initialize if storm id does not exist
        if storm_id not in models[model_names[i]].storm.keys() :
            models[model_names[i]].storm[storm_id] = dict()

        models[model_names[i]].storm[storm_id].update({
            timestamp : {
            "sample_sizes" : sample_sizes,
            "lat" : latitude,
            "long" : longitude,
            "wind_speed" : wind_speed,
            "intensity_forecast" : intensity_forecast,
            "track_forecast" : track_forecast,
            }
        })

In [ ]:
# Show the first 3 OFCL hurricane model errors for Hurricane Katrina 2005 on 28-08-2005/18:00:00
pprint(models['OFCL'].storm['AL122005'][datetime.datetime(2005, 8, 28, 18, 0)], indent = 8)

In [ ]:
# Show the first 5 records from Hurricane Katrina 2005 (AL122005)
dataset.query('storm_id == "AL122005"').head()

# Transform Data
The following code will tranform the hurricane best path data into objects that can be better manipulated for processing. to match between datasets, we will also create a storm_id dictionary to store storm names matched with ID's

In [ ]:
# Create hurricane class
class hurricane(object) : 
    def __init__(self, name, id) :
        # Set instance variables
        self.name = name
        self.id = id
        self.entries = dict()
        self.models = dict()
        
        return
    # Add hurricane track entry based on standard HURDAT2 format
    def add_entry(self, array) :
        entry = {
            array[0] : { # dateteime of entry
                'entry_time' : array[0], 
                'entry_id' : array[1],
                'entry_status' : array[2],
                'lat' : float(array[3][:-1]), # Convert to number from format '#.#N'
                'long' : float(array[4][:-1]), # Convert to number from format '#.#W'
                'max_wind' : float(array[5]),
                'min_pressure' : 980 if array[6] is None else float(array[6]), # Early records are -999 or None
                'wind_radii' :  array[7:], # Array based on HURDAT2 format
                'distance': 0,
                'direction': 0
            }
        }
        self.entries.update(entry)
        
        return
    # Add hurricane model errors
    def add_model(self, name, model) :
        self.models[name] = model
        
        return


    def update_dist_direc(self):
      t = pd.DataFrame(self.entries.values())
      dst = 0
      prev = (0,0)
      
      # For all latitude and longitude points of hurricane, calculate the angle of travel and distance
      for index,p in enumerate(zip(t['lat'], t['long'])):
          
          if prev == (0,0):
              prev = p
              continue 
          # Stores the distance into the DataFrame
          list(self.entries.values())[index]['distance'] = vc(prev,p).miles
          
          dLon = p[1] - prev[1];  
          temp = float(p[0]) # p[0] is a str?
          y_x = Math.sin(dLon) * Math.cos(temp);
          
          x_x = Math.cos(p[1]) * Math.sin(temp) - Math.sin(p[1]) * Math.cos(temp) * Math.cos(dLon);
          brng = Math.degrees(Math.atan2(y_x, x_x)) 
          if (brng < 0):
              brng+= 360;
          
          # Stores the angle of travel into the DataFrame
          list(self.entries.values())[index]['direction'] = brng
          # if self.id == 'AL122005' and index==2:
          if self.id == 'AL081994' and index==2:
            print(f'p[1]:{p[1]}')
            print(f'prev[1]:{prev[1]}')
            print(f'dLon:{dLon}')
            print(f'temp:{temp}')
            print(f'y_x:{y_x}')
            print(f'x_x:{x_x}')
            print(f'brng:{brng}')
          dst += vc(prev,p).miles
          prev = p

# Storm ID Key for matching between datasets
storm_ids = dict()
# Parse in hurricanes
hurricanes = dict()

print("Transforming HURDAT2 into objects . . .")
for index, entry in dataset.iterrows() :
    print("Transforming {}/{} entries from HURDAT2".format(index + 1, len(dataset)), end = "\r")
    # New hurricane
    if entry['storm_id'] not in hurricanes :
        hurricanes[entry['storm_id']] = hurricane(entry['storm_name'], entry['storm_id'])
        storm_ids[entry['storm_id']] = entry['storm_name']
    # Add entry to hurricane
    hurricanes[entry['storm_id']].add_entry(entry[2:])
print("\nDone!")

# Load Data
The following will finalize our preliminary data preparation by loading some of the errors into each hurricane object. Note that models start from the year 1970 and any hurricane before that has no previous model data.

In [ ]:
# Get all available model errors
# Load model errors into hurricanes
for id in storm_ids :
    for model in models :
        # Skip if this hurricane does not have the model
        if id not in models[model].storm :
            continue
        hurricanes[id].add_model(model, models[model].storm[id])
    hurricanes[id].update_dist_direc()

In [ ]:
#will test distance and direction of the bulk update vs individual update
t=pd.DataFrame(hurricanes['AL081994'].entries.values())
t

In [ ]:
t['dist']=0
t['direc']=0

In [ ]:
#testing for one hurricane
prev=(0,0)
for index,p in enumerate(zip(t['lat'], t['long'])):
  if prev == (0,0):
    prev = p
    print(f'index:{index},prev:{prev}')
    continue 
  # Stores the distance into the DataFrame
  t.at[index,'dist'] = vc(prev,p).miles

  dLon = p[1] - prev[1];  
  temp = float(p[0]) # p[0] is a str?
  y_x = Math.sin(dLon) * Math.cos(temp);

  x_x = Math.cos(p[1]) * Math.sin(temp) - Math.sin(p[1]) * Math.cos(temp) * Math.cos(dLon);
  brng = Math.degrees(Math.atan2(y_x, x_x)) 
  if (brng < 0):
    brng+= 360;
  t.at[index,'direc'] = brng
  if index==2:
    print(f'p[1]:{p[1]}')
    print(f'prev[1]:{prev[1]}')
    print(f'dLon:{dLon}')
    print(f'temp:{temp}')
    print(f'y_x:{y_x}')
    print(f'x_x:{x_x}')
    print(f'brng:{brng}')
  print(index,t.at[index,'direc'])
  prev = p

In [ ]:
t #to check that distance calculation and direction calculation from bulk vs individual

In [ ]:
models.keys()

# Feature Engineering & Data Augmentation
The following section will extract the relevant features and engineer each data point so that we can fit it into the model. Because the type of inputs are important, the features will be transformed based on the model architecture. This will also include data augmentation methods. The higher level architecture will be a deep learning recurrent neural network with LSTM and time distributed layers.

The current statistical baseline model using multivariate regression uses multiple predictors as input. According to Knaff 2013, the following predictors were calculated for their intensity model that were not included in the HURDAT2 database. These features can be calculated from the data loaded into our current object model.
1. Date Information
2. Zonal Speed Of The Storm (U) (kt)
3. Meridional Speed Of The Storm (V) (kt)
4. 12-h Change In Intensity (DVMX) (kt)

The shape on the input to the LSTM will be in a 3D array with the format [samples, timestamps, features]. We will intitially begin with 1 time step and evaluate more can benefit our model. The output requires a 5 day forecast and observations without track data 5 days in the future will not be used.

In [ ]:
def feature_extraction(timestep, previous) :
    '''
    PURPOSE: Calculate the features for a machine learning model within the context of hurricane-net
    METHOD: Use the predictors and the calculation methodology defined in Knaff 2013
    INPUT:  timestep - current dictionary of features in the hurricane object format
            previous - previous timestep dictionary of features in the hurricane object format
    OUTPUT: Dictionary of features
    
    timestep = {
      'lat' : float,
      'long' : float,
      'max-wind' : float,
      'entry-time' : datetime
    }
    '''
    features = {
        'lat' : timestep['lat'],
        'long' : timestep['long'],
        'max_wind' : timestep['max_wind'],
        'delta_wind' : (timestep['max_wind'] - previous['max_wind']) / # Calculated from track (12h)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 43200),
        'min_pressure' : timestep['min_pressure'], 
        'zonal_speed' : (timestep['lat'] - previous['lat'])/ # Calculated from track (per hour)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 3600),
        'meridonal_speed' : (timestep['long'] - previous['long'])/# Calculated from track (per hour)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 3600),
        'year' : timestep['entry_time'].year,
        'month' : timestep['entry_time'].month,
        'day' : timestep['entry_time'].day,
        'hour' : timestep['entry_time'].hour,
        'delta_pressure': (timestep['min_pressure'] - previous['min_pressure']) /
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 43200),
        'distance': timestep['distance'],
        'direction': timestep['direction']
    }
    return features
    
def storm_x_y(storm, timesteps = 1, lag = 24) :
    '''
    PURPOSE: Create independent and dependent samples for a machine learning model based on the timesteps
    METHOD: Use the HURDAT2 database and a hurricane object as defined in hurricane-net for feature extraction
    INPUT:  storm - hurricane object
            timesteps - (default = 1) number of timesteps to calculate
            include_none - (default = False) Boolean for including None in test data. Imputing function unavailable.
            lag - (default = 24) lag in hours for the dependent variables up to 5 days
    OUTPUT: Dictionary with independent (x) and dependent (y) values.
    '''
    x = []
    # Create testing data structure with a dictionary
    times = [time * lag for time in range(1, (120 // lag) + 1)] # Begin at lag hours with lag increments up to 120h inclusive
    y = dict([(time,[]) for time in times])
    
    # Sort by entry time
    entries = [entry[1] for entry in sorted(storm.entries.items())]
    
    for index in range(len(entries)) :
        if index < timesteps : # Flag for insufficient initial time steps
            continue

        # If we're not including None values, check to see if there will be any
        if None in [storm.entries.get(entries[index]['entry_time'] +
                                         datetime.timedelta(hours = future)) for future in times] : break
            
        # Calculate time steps and their features for independent values
        sample = []
        for step in range(timesteps) :
            # Training sample
            timestep = entries[index - step]
            previous = entries[index - step - 1]
            sample.append([timestep['entry_time']] + [[feature_extraction(timestep, previous)]])
        x.append(sample) # Add our constructed sample
        
        # Calculate time steps and their features for dependent values
        for future in times :
            timestep = storm.entries.get(entries[index]['entry_time'] + datetime.timedelta(hours = future))
            previous = storm.entries.get(entries[index]['entry_time'] + datetime.timedelta(hours = future - lag))
            
            if timestep and previous: 
                y[future].append(feature_extraction(timestep, previous))
            else :
                y[future].append(None)
    
    # Return output, if there is no output, return None.
    if len(x) == 0 :
        return None
    else:
        return {'x': x, 'y': y}
def shape(hurricanes, timesteps, remove_missing = True) :
    '''
    PURPOSE: Shape our data for input into machine learning models
    METHOD: Use a numpy array to shape into (samples, timesteps, features)
    INPUT:  hurricanes - dictionary of hurricane objects
            timesteps - number of timesteps for the shape
            remove_missing - boolean indicating whether the algorithm will disregard missing values
    OUTPUT: numpy array of shape (samples, timesteps, 11) where 11 is the number of predictors in a hurricane object
    '''
    x = []
    y = []
    lag = 24 # lag time in hours
    precision = np.float64 # defines the precision of our data type
    times = [time * lag for time in range(1, (120 // lag) + 1)] # Begin at lag hours with lag increments up to 120h inclusive
    count = 0
    for hurricane in hurricanes.values() :
        count += 1
        result = storm_x_y(hurricane, timesteps, lag)
        if result is None :
            continue
        # Extract only the values from the strom features using our specified precision
        hurricane_x = np.array(
            [[list(sample[1][0].values()) for sample in x] for x in result['x']],
            dtype = precision)
        hurricane_y = np.array(
            [[list(result['y'][time][index].values()) for time in times] for index in range(len(result['y'][lag]))],
            dtype = precision)
        # Disregard if algorithm requires no missing values
        if remove_missing :
            if (len(np.where(np.isnan(hurricane_x))[0]) > 0) or (len(np.where(np.isnan(hurricane_y))[0]) > 0) :
                continue
        # Add to our results
        x.extend(hurricane_x)
        y.extend(hurricane_y)
        print("Feature engineered {}/{} hurricanes for {} timestep(s)".format(count, len(hurricanes), timesteps), end = "\r")
    print("\nDone feature engineering hurricanes.")
    
    return {'x': np.array(x), 'y': np.array(y)}
def scaler(processed_data, hurricanes) :
    '''
    PURPOSE: Scale our data using the RobustScaler method from the sklearn library
    METHOD: Generate data using 1 timesteps and then remove the NaN or None types to use the scaler methods
    INPUT:  hurricanes - dictionary of hurricane objects
            processed_data - dictionary of x and y values of data produced by shape() function with no missing values
    OUTPUT: 1) Scaled processed_data using RobustScaler
            2) RobustScaler object fit with appropriate data
    '''
    print("Scaling Data . . . (1 timestep for unqiue data)")
    # Create our scaler
    unqiue_data = shape(hurricanes, timesteps = 1)
    x = np.reshape(unqiue_data['x'], (unqiue_data['x'].shape[0], -1))
    x = np.delete(x, np.where(np.isnan(x))[0], 0)
    scaler = RobustScaler()
    scaler.fit(x)
    
    # Scale our data
    for index in range(len(processed_data['x'])) :
        # Scale our x
        processed_data['x'][index] = scaler.transform(processed_data['x'][index])
        # Scale our y
        processed_data['y'][index] = scaler.transform(processed_data['y'][index])
    print("Done scaling.")
    return processed_data, scaler
# Finalize and scale procesed data into a dictionary
preprocessed_data = shape(hurricanes, timesteps = 5)
processed_data, scaler = scaler(preprocessed_data, hurricanes)

# Model Architecture

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.layers import LSTM, Dense, Input, Lambda, Attention, Layer
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np


class AttentionLayer(Layer):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.hidden_size = hidden_size

    def build(self, input_shape):
        self.context_vector = self.add_weight(shape=(self.hidden_size,),
                                              initializer='random_normal',
                                              trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        hidden_states, _ = inputs

        context_vector = tf.expand_dims(self.context_vector, axis=0)
        context_vector = tf.repeat(context_vector, tf.shape(hidden_states)[0], axis=0)

        attention_weights = tf.einsum('ijk,ik->ij', hidden_states, context_vector)
        attention_weights = tf.keras.layers.Activation('softmax')(attention_weights)

        weighted_sum = tf.einsum('ij,ijk->ik', attention_weights, hidden_states)

        return weighted_sum



In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

def attention_layer(inputs):
    hidden_states, context_vector = inputs
    
    hidden_size = int(hidden_states.shape[2])
    
    # Reshape context vector to perform element-wise multiplication
    context_vector = Dense(hidden_size)(context_vector)
    context_vector = tf.repeat(context_vector, tf.shape(hidden_states)[1], axis=1)
    
    # Attention mechanism
    attention_weights = tf.keras.layers.Attention()([hidden_states, context_vector])
    attention_weights = tf.keras.layers.Activation('softmax')(attention_weights)
    
    # Weighted sum of hidden states
    weighted_sum = tf.reduce_sum(attention_weights * hidden_states, axis=1)
    
    return weighted_sum

def build_model(input_shape, hidden_units, output_dim, dropout_rate=0.15, recurrent_dropout_rate=0.15):
    inputs = Input(shape=input_shape)
    lstm_output = LSTM(hidden_units, return_sequences=True, dropout=dropout_rate, recurrent_dropout=recurrent_dropout_rate)(inputs)
    attention_output = AttentionLayer(hidden_units)([lstm_output, lstm_output])
    time_distributed_output = TimeDistributed(Dense(output_dim))(attention_output)

    model = Model(inputs=inputs, outputs=time_distributed_output)
    return model

def train_model(X_train, X_test, y_train, y_test, output_dim, n_epochs):
    input_shape = X_train.shape[1:]
    hidden_units = 128

    model = build_model(input_shape, hidden_units, output_dim)
    model.summary()

    model.compile(optimizer='adam', loss='mean_squared_error')

    loss = []
    val_loss = []
    for epoch in range(n_epochs):
        # Early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        print(f"Epoch {epoch+1}/{n_epochs}")
        history = model.fit(X_train, y_train, epochs=1, batch_size=512, validation_data=(X_test, y_test), verbose=1)
        print("Loss:", history.history['loss'][0])
        loss.append(history.history['loss'][0])
        print("Validation Loss:", history.history['val_loss'][0])
        val_loss.append(history.history['val_loss'][0])
        print()

    return model, history, loss, val_loss

# Create our cross-validation data structure
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(processed_data['x'], processed_data['y'], train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# Define the features to train for
features = [2, 0, 1]  # Specify the indices of the features to train for

# Train multiple models
models = []
histories = []
losses = []
val_losses = []
n_epochs = 500

for feature_idx in features:
    y_train_feature = np.array([[[features[feature_idx]] for features in y] for y in y_train], dtype=np.float64)
    y_valid_feature = np.array([[[features[feature_idx]] for features in y] for y in y_valid], dtype=np.float64)
    
    model, history, loss, val_loss = train_model(X_train, X_valid, y_train_feature, y_valid_feature, output_dim=1, n_epochs=n_epochs)
    
    models.append(model)
    histories.append(history)
    losses.append(loss)
    val_losses.append(val_loss)

# Access the history of each model
for i, history in enumerate(histories):
    print(f"History for Model {i+1}")
    print("Loss:", history.history['loss'])
    print("Validation Loss:", history.history['val_loss'])
    print()


In [ ]:
# Predict values
wind_predictions = models[0].predict(X_test)
lat_predictions = models[1].predict(X_test)
long_predictions = models[2].predict(X_test)

In [ ]:
# Plot loss over time
plt.figure(figsize=(10, 6))
for i, loss in enumerate(losses):
    plt.plot(loss, label=f"Model {i+1}")
plt.title("Loss Over Time")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot validation loss over time
plt.figure(figsize=(10, 6))
for i, history in enumerate(histories):
    plt.plot(val_loss, label=f"Model {i+1}")
plt.title("Validation Loss Over Time")
plt.xlabel("Epochs")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()

In [ ]:
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
wind_history = asarray(history[0])
wind_predictions = asarray(wind_predictions)

lat_history = asarray(history[1])
lat_predictions = asarray(lat_predictions)

long_history = asarray(history[2])
long_predictions = asarray(long_predictions)

# save to csv file
savetxt('wind_hist.csv', wind_history, delimiter=',')
savetxt('wind_preds.csv', wind_predictions, delimiter=',')
savetxt('lat_hist.csv', lat_history, delimiter=',')
savetxt('lat_predictions.csv', lat_predictions, delimiter=',')
savetxt('long_hist.csv', long_history, delimiter=',')
savetxt('long_preds_hist.csv', long_predictions, delimiter=',')